# GARCH Volatility Forecaster

This notebook demonstrates GARCH-based volatility modeling and forecasting.

**Models implemented:**
- GARCH(1,1) - Standard model
- EGARCH(1,1) - Exponential GARCH (captures asymmetry)
- GJR-GARCH(1,1) - Threshold GARCH (leverage effect)

Author: Avni Derashree

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from src.data_loader import prepare_volatility_data, calculate_realized_volatility
from src.garch_models import (
    fit_garch, fit_multiple_garch_models, get_conditional_volatility,
    model_comparison_table, stationarity_tests, arch_effect_test
)
from src.forecasting import forecast_volatility, compare_with_vix, term_structure_forecast
from src.visualization import (
    plot_return_and_volatility, plot_garch_diagnostics,
    plot_forecast, plot_model_comparison, plot_vix_comparison
)

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

## 1. Load Market Data

In [ ]:
# Load SPY data (S&P 500 ETF)
ticker = "SPY"
prices, returns, realized_vol, vix = prepare_volatility_data(ticker, period="5y")

print(f"Asset: {ticker}")
print(f"Data points: {len(returns)}")
print(f"Date range: {returns.index[0].date()} to {returns.index[-1].date()}")

In [ ]:
# Return statistics
stats = {
    'Mean (daily)': f"{returns.mean():.4%}",
    'Std (daily)': f"{returns.std():.4%}",
    'Annualized Vol': f"{returns.std() * np.sqrt(252):.2%}",
    'Skewness': f"{returns.skew():.3f}",
    'Kurtosis': f"{returns.kurtosis():.3f}",
    'Min': f"{returns.min():.4%}",
    'Max': f"{returns.max():.4%}"
}
pd.DataFrame(stats, index=['Value']).T

## 2. Pre-Modeling Diagnostics

Before fitting GARCH, we verify:
1. **Stationarity** - Returns should be stationary
2. **ARCH effects** - Volatility clustering should be present

In [ ]:
# Stationarity tests
stat_tests = stationarity_tests(returns)

print("Stationarity Tests:")
print(f"  ADF: stat={stat_tests['ADF'][0]:.3f}, p={stat_tests['ADF'][1]:.4f}")
print(f"  KPSS: stat={stat_tests['KPSS'][0]:.3f}, p={stat_tests['KPSS'][1]:.4f}")

# ARCH effect test
arch_stat, arch_pval = arch_effect_test(returns, lags=10)
print(f"\nARCH-LM Test: stat={arch_stat:.2f}, p={arch_pval:.4f}")
print(f"  → {'ARCH effects present ✓' if arch_pval < 0.05 else 'No ARCH effects ✗'}")

## 3. Fit GARCH Models

In [ ]:
# Fit multiple GARCH variants
results = fit_multiple_garch_models(returns, models=['GARCH', 'EGARCH', 'GJR-GARCH'])

# Display comparison
model_comparison_table(results)

In [ ]:
# Select best model (by AIC)
best_name = min(results, key=lambda x: results[x].aic)
best_result = results[best_name]

print(f"Best Model: {best_result.model_name}")
print(f"\nParameter Interpretation:")
print(f"  α = {best_result.alpha:.4f} (shock impact)")
print(f"  β = {best_result.beta:.4f} (persistence)")
print(f"  Persistence = {best_result.persistence:.4f}")
print(f"  Half-life = {best_result.half_life:.1f} days")
print(f"  Long-run vol = {best_result.unconditional_vol:.2%}")

## 4. Model Diagnostics

In [ ]:
# GARCH diagnostics plots
fig = plot_garch_diagnostics(best_result, figsize=(14, 10))
plt.show()

## 5. Volatility Comparison

In [ ]:
# Get conditional volatility
garch_vol = get_conditional_volatility(best_result)

# Plot returns and volatility
fig = plot_return_and_volatility(returns, garch_vol, realized_vol, vix, figsize=(14, 10))
plt.show()

In [ ]:
# Compare all models
fig = plot_model_comparison(results, realized_vol, figsize=(14, 8))
plt.show()

## 6. VIX Comparison

In [ ]:
# Compare GARCH with VIX
vix_metrics = compare_with_vix(garch_vol, vix)

print("GARCH vs VIX Comparison:")
for metric, value in vix_metrics.items():
    if 'Correlation' in metric or 'Ratio' in metric:
        print(f"  {metric}: {value:.3f}")
    else:
        print(f"  {metric}: {value*100:.2f}%")

In [ ]:
# VIX scatter plot
fig = plot_vix_comparison(garch_vol, vix, figsize=(14, 6))
plt.show()

## 7. Volatility Forecasting

In [ ]:
# Generate 21-day forecast
forecast = forecast_volatility(best_result, horizon=21, confidence=0.95)

# Display forecast
forecast_df = forecast.to_dataframe()
forecast_df['Forecast %'] = forecast_df['Forecast'] * 100
forecast_df[['Forecast %', 'Lower', 'Upper']].head(10)

In [ ]:
# Plot forecast
fig = plot_forecast(garch_vol, forecast, realized_vol, figsize=(12, 6))
plt.show()

In [ ]:
# Volatility term structure
term_structure = term_structure_forecast(best_result, horizons=[1, 5, 10, 21, 63, 126, 252])
term_structure['Volatility %'] = term_structure['Volatility'] * 100
term_structure[['Term', 'Volatility %']]

## 8. Key Insights

In [ ]:
print("🔍 KEY FINDINGS")
print("=" * 50)

print(f"\n1. Best Model: {best_result.model_name}")
print(f"   AIC: {best_result.aic:.1f}")

print(f"\n2. Volatility Persistence: {best_result.persistence:.4f}")
if best_result.persistence > 0.95:
    print("   → High persistence: shocks decay slowly")

print(f"\n3. VIX Correlation: {vix_metrics['Correlation']:.3f}")
if vix_metrics['Correlation'] > 0.7:
    print("   → Strong correlation: model tracks market fear")

print(f"\n4. Current Volatility:")
print(f"   GARCH: {garch_vol.iloc[-1]*100:.2f}%")
print(f"   VIX: {vix.iloc[-1]*100:.2f}%")

print(f"\n5. 21-Day Forecast: {forecast.point_forecast[-1]*100:.2f}%")